In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import time
import json
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm_notebook

HEADERS = {'User-Agent': 'Mozilla/5.0', 'Accept': 'text/html,application/xhtml+xml,application/xml'}

In [3]:
import pandas as pd
urls_df = pd.read_csv('./archive/UE-NY-rest-urls.csv')
urls_df.head()

,url,city
0,https://www.ubereats.com/new-york/food-deliver...,new-york
1,https://www.ubereats.com/new-york/food-deliver...,new-york
2,https://www.ubereats.com/new-york/food-deliver...,new-york
3,https://www.ubereats.com/new-york/food-deliver...,new-york
4,https://www.ubereats.com/new-york/food-deliver...,new-york


In [4]:
def get_rest_details(restaurant_url):
    start_time = time.time()
    restaurant_obj = {}
    try:
        req = Request(restaurant_url, headers=HEADERS)
        webpage = urlopen(req, timeout=20).read()
        soup = BeautifulSoup(webpage, 'html.parser')
        restaurant_obj = json.loads(soup.find("script", type="application/ld+json").text)
    except Exception as e:
        print(f"Restaurant error: {e} url: {restaurant_url}")
    return restaurant_obj

In [5]:
r_url = "https://www.ubereats.com/new-york/food-delivery/vive-la-crepe-colombus-%26-68th-st/E5pHJPJrRV64y20haSxCUQ"
obj = get_rest_details(r_url)
obj

{'@context': 'http://schema.org',
 '@id': 'https://www.ubereats.com/new-york/food-delivery/vive-la-crepe-colombus-%26-68th-st/E5pHJPJrRV64y20haSxCUQ',
 '@type': 'Restaurant',
 'address': {'@type': 'PostalAddress',
  'addressCountry': 'US',
  'addressLocality': 'New York',
  'addressRegion': 'NY',
  'postalCode': '10023',
  'streetAddress': '189 Columbus Ave'},
 'aggregateRating': {'@type': 'AggregateRating',
  'ratingValue': 4.5,
  'reviewCount': '19'},
 'geo': {'@type': 'GeoCoordinates',
  'latitude': 40.7750114,
  'longitude': -73.9803166},
 'image': ['https://d1ralsognjng37.cloudfront.net/ab7cd8be-4e13-4210-a4af-52a6432d2871.jpeg',
  'https://d1ralsognjng37.cloudfront.net/cf8317f1-2bca-4f03-a15a-682d5bffdaba.jpeg',
  'https://d1ralsognjng37.cloudfront.net/d217f7ce-0f49-4fe9-8c35-9ffdbdb44c29.jpeg',
  'https://d1ralsognjng37.cloudfront.net/a6ce24ca-cd66-445f-a979-d141ce2a919c.jpeg',
  'https://d1ralsognjng37.cloudfront.net/703c9b20-1d2f-4d3d-b578-3ce77c415da3.jpeg',
  'https://d1rals

In [ ]:
def get_inner_object(string):
    obj = {}
    if string != None:
        obj = json.loads(string.replace('\'', "\""))
    return obj

def scrap_city_restaurants(cityname):
    urls_df = pd.read_csv(cityname+".csv")
    num_restaurants = urls_df.shape[0]
    detais_df = pd.DataFrame(columns=
                             ['name',
                              'city', 
                              'cuisine', 
                              'avgRating', 
                              'numReviews', 
                              'priceRange', 
                              'latitude',
                              'longitude'
                              'telephone', 
                              'url', 
                              'postalCode',
                              'streetAddress',
                              'addressLocality', 
                              'addressRegion', 
                              'openingHoursSpecification',
                              'addressString',
                              'geoString',
                              'ratingString'])
    
    for i in tqdm(range(5), desc=cityname):
        curr_rest = urls_df.iloc[i]     
        try:
            obj = get_rest_details(curr_rest.url, error_rests)
            if obj == None:
                raise ValueError(f'Error: Obj was none. City: {cityname})
            
            rating_obj = get_inner_object(obj.get('aggregateRating'))
            address_obj = get_inner_object(obj.get('address'))
            geo_obj = get_inner_object(obj.get('geo'))
            
            dict1 = {
                        'name': obj['name'],
                        'city': curr_rest.city
                        'cuisine': obj.get('servesCuisine')
                        'avgRating': rating_obj.get('ratingValue'),
                        'numReviews': rating_obj.get('reviewCount'),
                        'priceRange': obj.get('priceRange'),
                        'latitude': geo_obj.get('latitude'),
                        'longitude': geo_obj.get('longitude'),
                        'telephone': obj.get('telephone'),
                        'url': curr_rest.url,
                        'postalCode': address_obj.get('postalCode'),
                        'streetAddress': address_obj.get('streetAddress'),
                        'addressLocality': address_obj.get('addressLocality'),
                        'addressRegion': address_obj.get('addressRegion'),
                        'openingHoursSpecification': obj.get('openingHoursSpecification'),
                        'addressString': obj.get('address'),
                        'geoString': obj.get('geo'),
                        'ratingString': obj.get('aggregateRating')
                    }
                                 
            
        except Exception as e:
            print(f"Error: {e} url: {curr_rest.url}")

In [ ]:
def store_city_restaurants(df):
    

In [6]:
str(None)

'None'